In [2]:
import svgwrite
import json
ARROW_HEIGHT=10
PROP_LEFT_MARGIN=20
PROP_BLOCK_HEIGHT=30
PROP_BLOCK_WIDTH=70
PROP_BLOCK_MARGIN=30
PROP_BLOCK_TEXT_HEIGHT=20
PROP_BLOCK_TEXT_MARGIN=5

def addPropBlock(dwg,x,y, text='', info='', leader=False):
    g=dwg.g(class_='Proposer')
    if leader:
        g.add(dwg.rect((PROP_LEFT_MARGIN+x,y),(PROP_BLOCK_WIDTH, PROP_BLOCK_HEIGHT),rx=3,ry=3,class_='ProposerLeader'))
    else:
        g.add(dwg.rect((PROP_LEFT_MARGIN+x,y),(PROP_BLOCK_WIDTH, PROP_BLOCK_HEIGHT),rx=3,ry=3,class_='Proposer'))
    if text!='':
        g.add(dwg.text(text,(PROP_LEFT_MARGIN+PROP_BLOCK_TEXT_MARGIN+x,y+PROP_BLOCK_TEXT_HEIGHT)))
    if info!='':
        g.set_desc(title=info)
    dwg.add(g)

# def addPropParentArrow(dwg,x,y):
#     dwg.add(dwg.line((PROP_LEFT_MARGIN+x+PROP_BLOCK_WIDTH/2,y),
#                      (PROP_LEFT_MARGIN+x+PROP_BLOCK_WIDTH/2, y-PROP_BLOCK_MARGIN+ARROW_HEIGHT), 
#                      stroke="#000000", stroke_width="1", marker_end="url(#ToProposerParent)"))


In [3]:

def parseBlockchainDump(dump):
    def h256ToHex(h):
        return '{:032X}{:032X}'.format(h[0],h[1])
    d=json.loads(dump)
    leader_set=set()
    for k,v in d['leader_nodes'].items():
        leader_set.add(h256ToHex(v))
    prop_nodes = []
    max_layer_width = 0
    for layer in d['prop_nodes']:
        l = []
        for h in layer:
            l.append(h256ToHex(h))
        max_layer_width = max(max_layer_width, len(l))
        prop_nodes.append(l)
    prop_votes = {}
    for h,level,leader_status,votes in d['proposer_node_data']:
        prop_votes[h256ToHex(h)]=int(votes)
    edges = {}
    for h1,h2,edge_type in d['edges']:
        if type(edge_type) is str:
            edges[(h256ToHex(h1),h256ToHex(h2))]=edge_type
        elif type(edge_type) is dict:
            t=next(iter(edge_type))
            edges[(h256ToHex(h1),h256ToHex(h2))]='{}({})'.format(t,edge_type[t])
            
    ret = {'leader_set':leader_set, 'prop_nodes': prop_nodes, 
           'max_layer_width':max_layer_width, 'prop_votes':prop_votes, 
           'edges':edges}
    return ret

In [4]:
# a toy blockchain dump
dump='''{"edges":[[[0,0],[16309190651391274726615603873253518193,74615886398085577132179308356420263538],"TransactionFromProposerParent"],[[16309190651391274726615603873253518193,74615886398085577132179308356420263538],[0,0],"TransactionToProposerParent"],[[0,0],[203416634904130912947837948206129956701,84667328377361762844812902622586151209],"TransactionFromProposerParent"],[[203416634904130912947837948206129956701,84667328377361762844812902622586151209],[0,0],"TransactionToProposerParent"],[[0,0],[127631816784194221745890895948874516060,98040460094865379368012538286735519919],"TransactionFromProposerParent"],[[127631816784194221745890895948874516060,98040460094865379368012538286735519919],[0,0],"TransactionToProposerParent"],[[0,0],[13666062443490743129725895377544875000,59332087016073312160008192530063055542],"TransactionFromProposerParent"],[[13666062443490743129725895377544875000,59332087016073312160008192530063055542],[0,0],"TransactionToProposerParent"],[[0,0],[113140849040239369915023536159678568992,215910713632893116095956307163399606234],"TransactionFromProposerParent"],[[113140849040239369915023536159678568992,215910713632893116095956307163399606234],[0,0],"TransactionToProposerParent"],[[0,0],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"ProposerFromProposerParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[0,0],"ProposerToProposerParent"],[[16309190651391274726615603873253518193,74615886398085577132179308356420263538],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],{"ProposerFromTransactionReference":0}],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[16309190651391274726615603873253518193,74615886398085577132179308356420263538],{"ProposerToTransactionReference":0}],[[203416634904130912947837948206129956701,84667328377361762844812902622586151209],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],{"ProposerFromTransactionReference":1}],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[203416634904130912947837948206129956701,84667328377361762844812902622586151209],{"ProposerToTransactionReference":1}],[[127631816784194221745890895948874516060,98040460094865379368012538286735519919],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],{"ProposerFromTransactionReference":2}],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[127631816784194221745890895948874516060,98040460094865379368012538286735519919],{"ProposerToTransactionReference":2}],[[13666062443490743129725895377544875000,59332087016073312160008192530063055542],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],{"ProposerFromTransactionReference":3}],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[13666062443490743129725895377544875000,59332087016073312160008192530063055542],{"ProposerToTransactionReference":3}],[[113140849040239369915023536159678568992,215910713632893116095956307163399606234],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],{"ProposerFromTransactionReference":4}],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[113140849040239369915023536159678568992,215910713632893116095956307163399606234],{"ProposerToTransactionReference":4}],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[196222927150313727623849423392830757607,48601561847489824114585381840201303570],"VoterFromProposerParentAndVote"],[[196222927150313727623849423392830757607,48601561847489824114585381840201303570],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,1],[196222927150313727623849423392830757607,48601561847489824114585381840201303570],"VoterFromVoterParent"],[[196222927150313727623849423392830757607,48601561847489824114585381840201303570],[0,1],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[287043098010830812125523790819448648783,131487889733446119788561996676586206793],"VoterFromProposerParentAndVote"],[[287043098010830812125523790819448648783,131487889733446119788561996676586206793],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,2],[287043098010830812125523790819448648783,131487889733446119788561996676586206793],"VoterFromVoterParent"],[[287043098010830812125523790819448648783,131487889733446119788561996676586206793],[0,2],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[17914026366725359034877494502710405716,219442844074318402922155726089159906818],"VoterFromProposerParentAndVote"],[[17914026366725359034877494502710405716,219442844074318402922155726089159906818],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,3],[17914026366725359034877494502710405716,219442844074318402922155726089159906818],"VoterFromVoterParent"],[[17914026366725359034877494502710405716,219442844074318402922155726089159906818],[0,3],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[48201290777533410771139438958502714464,27213940389256323001680455193029967216],"VoterFromProposerParentAndVote"],[[48201290777533410771139438958502714464,27213940389256323001680455193029967216],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,4],[48201290777533410771139438958502714464,27213940389256323001680455193029967216],"VoterFromVoterParent"],[[48201290777533410771139438958502714464,27213940389256323001680455193029967216],[0,4],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[334952087411119706473059337547418793746,229657452203028483274315913153769090502],"VoterFromProposerParentAndVote"],[[334952087411119706473059337547418793746,229657452203028483274315913153769090502],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,5],[334952087411119706473059337547418793746,229657452203028483274315913153769090502],"VoterFromVoterParent"],[[334952087411119706473059337547418793746,229657452203028483274315913153769090502],[0,5],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[211196780309865720881828649784815609313,45125330659475175485776413142186369343],"VoterFromProposerParentAndVote"],[[211196780309865720881828649784815609313,45125330659475175485776413142186369343],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,6],[211196780309865720881828649784815609313,45125330659475175485776413142186369343],"VoterFromVoterParent"],[[211196780309865720881828649784815609313,45125330659475175485776413142186369343],[0,6],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[262339210057093813629912155764713464083,4128170806167588733999200694047488729],"VoterFromProposerParentAndVote"],[[262339210057093813629912155764713464083,4128170806167588733999200694047488729],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,7],[262339210057093813629912155764713464083,4128170806167588733999200694047488729],"VoterFromVoterParent"],[[262339210057093813629912155764713464083,4128170806167588733999200694047488729],[0,7],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[133620542141301101600462131143722368744,154310728365235349425042104278799319053],"VoterFromProposerParentAndVote"],[[133620542141301101600462131143722368744,154310728365235349425042104278799319053],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,8],[133620542141301101600462131143722368744,154310728365235349425042104278799319053],"VoterFromVoterParent"],[[133620542141301101600462131143722368744,154310728365235349425042104278799319053],[0,8],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[243302063577874611527669088163318764758,83471547100651815018813919161061732632],"VoterFromProposerParentAndVote"],[[243302063577874611527669088163318764758,83471547100651815018813919161061732632],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,9],[243302063577874611527669088163318764758,83471547100651815018813919161061732632],"VoterFromVoterParent"],[[243302063577874611527669088163318764758,83471547100651815018813919161061732632],[0,9],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[217358329842609508847039961830287659919,133479465929730160242868124882031279602],"VoterFromProposerParentAndVote"],[[217358329842609508847039961830287659919,133479465929730160242868124882031279602],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"VoterToProposerParentAndVote"],[[0,10],[217358329842609508847039961830287659919,133479465929730160242868124882031279602],"VoterFromVoterParent"],[[217358329842609508847039961830287659919,133479465929730160242868124882031279602],[0,10],"VoterToVoterParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[287097079470211302833715110561734626650,25926857114035119223559103865408553040],"TransactionFromProposerParent"],[[287097079470211302833715110561734626650,25926857114035119223559103865408553040],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"TransactionToProposerParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[112843536655743373806773716133998813328,299687879528442309170905353923949392757],"TransactionFromProposerParent"],[[112843536655743373806773716133998813328,299687879528442309170905353923949392757],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"TransactionToProposerParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[70492910119632306246052673805011834441,53517089152114679239997810090471361907],"TransactionFromProposerParent"],[[70492910119632306246052673805011834441,53517089152114679239997810090471361907],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"TransactionToProposerParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[135234043505129978719737467984837497597,174040853874324827229657708681579943514],"TransactionFromProposerParent"],[[135234043505129978719737467984837497597,174040853874324827229657708681579943514],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"TransactionToProposerParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[188852550260518993802415706584420624711,301812328892229109507515401587485851177],"TransactionFromProposerParent"],[[188852550260518993802415706584420624711,301812328892229109507515401587485851177],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"TransactionToProposerParent"],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"ProposerFromProposerParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"ProposerToProposerParent"],[[287097079470211302833715110561734626650,25926857114035119223559103865408553040],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],{"ProposerFromTransactionReference":0}],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[287097079470211302833715110561734626650,25926857114035119223559103865408553040],{"ProposerToTransactionReference":0}],[[112843536655743373806773716133998813328,299687879528442309170905353923949392757],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],{"ProposerFromTransactionReference":1}],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[112843536655743373806773716133998813328,299687879528442309170905353923949392757],{"ProposerToTransactionReference":1}],[[70492910119632306246052673805011834441,53517089152114679239997810090471361907],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],{"ProposerFromTransactionReference":2}],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[70492910119632306246052673805011834441,53517089152114679239997810090471361907],{"ProposerToTransactionReference":2}],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],"ProposerFromProposerParent"],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"ProposerToProposerParent"],[[287097079470211302833715110561734626650,25926857114035119223559103865408553040],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],{"ProposerFromTransactionReference":0}],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[287097079470211302833715110561734626650,25926857114035119223559103865408553040],{"ProposerToTransactionReference":0}],[[112843536655743373806773716133998813328,299687879528442309170905353923949392757],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],{"ProposerFromTransactionReference":1}],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[112843536655743373806773716133998813328,299687879528442309170905353923949392757],{"ProposerToTransactionReference":1}],[[70492910119632306246052673805011834441,53517089152114679239997810090471361907],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],{"ProposerFromTransactionReference":2}],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[70492910119632306246052673805011834441,53517089152114679239997810090471361907],{"ProposerToTransactionReference":2}],[[135234043505129978719737467984837497597,174040853874324827229657708681579943514],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],{"ProposerFromTransactionReference":3}],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[135234043505129978719737467984837497597,174040853874324827229657708681579943514],{"ProposerToTransactionReference":3}],[[188852550260518993802415706584420624711,301812328892229109507515401587485851177],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],{"ProposerFromTransactionReference":4}],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[188852550260518993802415706584420624711,301812328892229109507515401587485851177],{"ProposerToTransactionReference":4}],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[193851307777460516918356531339904192027,165125033430926100122698488439366290008],"VoterFromProposerParentAndVote"],[[193851307777460516918356531339904192027,165125033430926100122698488439366290008],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParentAndVote"],[[196222927150313727623849423392830757607,48601561847489824114585381840201303570],[193851307777460516918356531339904192027,165125033430926100122698488439366290008],"VoterFromVoterParent"],[[193851307777460516918356531339904192027,165125033430926100122698488439366290008],[196222927150313727623849423392830757607,48601561847489824114585381840201303570],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[12937800391536231908324647127638186520,66280795779544227376177073920337078425],"VoterFromProposerParentAndVote"],[[12937800391536231908324647127638186520,66280795779544227376177073920337078425],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParentAndVote"],[[287043098010830812125523790819448648783,131487889733446119788561996676586206793],[12937800391536231908324647127638186520,66280795779544227376177073920337078425],"VoterFromVoterParent"],[[12937800391536231908324647127638186520,66280795779544227376177073920337078425],[287043098010830812125523790819448648783,131487889733446119788561996676586206793],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[125101350444128636945639187339273605295,310123101311045465600267532061795828904],"VoterFromProposerParentAndVote"],[[125101350444128636945639187339273605295,310123101311045465600267532061795828904],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParentAndVote"],[[17914026366725359034877494502710405716,219442844074318402922155726089159906818],[125101350444128636945639187339273605295,310123101311045465600267532061795828904],"VoterFromVoterParent"],[[125101350444128636945639187339273605295,310123101311045465600267532061795828904],[17914026366725359034877494502710405716,219442844074318402922155726089159906818],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[209247141366778780182390572487884510544,72710930124973944574458366969623903493],"VoterFromProposerParentAndVote"],[[209247141366778780182390572487884510544,72710930124973944574458366969623903493],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParentAndVote"],[[48201290777533410771139438958502714464,27213940389256323001680455193029967216],[209247141366778780182390572487884510544,72710930124973944574458366969623903493],"VoterFromVoterParent"],[[209247141366778780182390572487884510544,72710930124973944574458366969623903493],[48201290777533410771139438958502714464,27213940389256323001680455193029967216],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[169582566327277687462980578090338146030,202615754517709453024748782759834084329],"VoterFromProposerParentAndVote"],[[169582566327277687462980578090338146030,202615754517709453024748782759834084329],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParentAndVote"],[[334952087411119706473059337547418793746,229657452203028483274315913153769090502],[169582566327277687462980578090338146030,202615754517709453024748782759834084329],"VoterFromVoterParent"],[[169582566327277687462980578090338146030,202615754517709453024748782759834084329],[334952087411119706473059337547418793746,229657452203028483274315913153769090502],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[136624341944058342936762535411176582562,123747836904407515690884885245025584803],"VoterFromProposerParentAndVote"],[[136624341944058342936762535411176582562,123747836904407515690884885245025584803],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParentAndVote"],[[211196780309865720881828649784815609313,45125330659475175485776413142186369343],[136624341944058342936762535411176582562,123747836904407515690884885245025584803],"VoterFromVoterParent"],[[136624341944058342936762535411176582562,123747836904407515690884885245025584803],[211196780309865720881828649784815609313,45125330659475175485776413142186369343],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[218716363703852801541862396078244676608,221872901472969775451833903017499552185],"VoterFromProposerParentAndVote"],[[218716363703852801541862396078244676608,221872901472969775451833903017499552185],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParentAndVote"],[[262339210057093813629912155764713464083,4128170806167588733999200694047488729],[218716363703852801541862396078244676608,221872901472969775451833903017499552185],"VoterFromVoterParent"],[[218716363703852801541862396078244676608,221872901472969775451833903017499552185],[262339210057093813629912155764713464083,4128170806167588733999200694047488729],"VoterToVoterParent"],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[216764452460743606311030478227486484923,106930846498423695235466046775012050368],"VoterFromProposerParentAndVote"],[[216764452460743606311030478227486484923,106930846498423695235466046775012050368],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],"VoterToProposerParentAndVote"],[[133620542141301101600462131143722368744,154310728365235349425042104278799319053],[216764452460743606311030478227486484923,106930846498423695235466046775012050368],"VoterFromVoterParent"],[[216764452460743606311030478227486484923,106930846498423695235466046775012050368],[133620542141301101600462131143722368744,154310728365235349425042104278799319053],"VoterToVoterParent"],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[104447683660539520465595759369635189633,281072176520243182634918095165554116675],"VoterFromProposerParentAndVote"],[[104447683660539520465595759369635189633,281072176520243182634918095165554116675],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],"VoterToProposerParentAndVote"],[[243302063577874611527669088163318764758,83471547100651815018813919161061732632],[104447683660539520465595759369635189633,281072176520243182634918095165554116675],"VoterFromVoterParent"],[[104447683660539520465595759369635189633,281072176520243182634918095165554116675],[243302063577874611527669088163318764758,83471547100651815018813919161061732632],"VoterToVoterParent"],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[309479661309386212115965602171518273580,312006182838606330110754301785646750660],"VoterFromProposerParentAndVote"],[[309479661309386212115965602171518273580,312006182838606330110754301785646750660],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],"VoterToProposerParentAndVote"],[[217358329842609508847039961830287659919,133479465929730160242868124882031279602],[309479661309386212115965602171518273580,312006182838606330110754301785646750660],"VoterFromVoterParent"],[[309479661309386212115965602171518273580,312006182838606330110754301785646750660],[217358329842609508847039961830287659919,133479465929730160242868124882031279602],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[169364742848526736697216141342124312999,126145157880518463053192077034749149436],"TransactionFromProposerParent"],[[169364742848526736697216141342124312999,126145157880518463053192077034749149436],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"TransactionToProposerParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[196935908130825214331027925217695769814,141572546089812697211828151672257505424],"TransactionFromProposerParent"],[[196935908130825214331027925217695769814,141572546089812697211828151672257505424],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"TransactionToProposerParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[192980804815934475414241153111321128041,67988076356156252877453577212225517225],"TransactionFromProposerParent"],[[192980804815934475414241153111321128041,67988076356156252877453577212225517225],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"TransactionToProposerParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[338233728719313876516491589050094215276,57721423586908637536166780724712872734],"TransactionFromProposerParent"],[[338233728719313876516491589050094215276,57721423586908637536166780724712872734],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"TransactionToProposerParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"ProposerFromProposerParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"ProposerToProposerParent"],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],{"ProposerFromProposerReference":1}],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[334036293486207965471121892248981158011,119198624920124930478378759960496660278],{"ProposerToProposerReference":1}],[[169364742848526736697216141342124312999,126145157880518463053192077034749149436],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],{"ProposerFromTransactionReference":0}],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[169364742848526736697216141342124312999,126145157880518463053192077034749149436],{"ProposerToTransactionReference":0}],[[196935908130825214331027925217695769814,141572546089812697211828151672257505424],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],{"ProposerFromTransactionReference":1}],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[196935908130825214331027925217695769814,141572546089812697211828151672257505424],{"ProposerToTransactionReference":1}],[[192980804815934475414241153111321128041,67988076356156252877453577212225517225],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],{"ProposerFromTransactionReference":2}],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[192980804815934475414241153111321128041,67988076356156252877453577212225517225],{"ProposerToTransactionReference":2}],[[338233728719313876516491589050094215276,57721423586908637536166780724712872734],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],{"ProposerFromTransactionReference":3}],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[338233728719313876516491589050094215276,57721423586908637536166780724712872734],{"ProposerToTransactionReference":3}],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[302252672319357169528154936146331567083,122555437337793757347265884757286863129],"VoterFromProposerParent"],[[302252672319357169528154936146331567083,122555437337793757347265884757286863129],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParent"],[[193851307777460516918356531339904192027,165125033430926100122698488439366290008],[302252672319357169528154936146331567083,122555437337793757347265884757286863129],"VoterFromVoterParent"],[[302252672319357169528154936146331567083,122555437337793757347265884757286863129],[193851307777460516918356531339904192027,165125033430926100122698488439366290008],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[224393777323635580607144898021248152058,249955563744089122721854621298071612254],"VoterFromProposerParent"],[[224393777323635580607144898021248152058,249955563744089122721854621298071612254],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParent"],[[12937800391536231908324647127638186520,66280795779544227376177073920337078425],[224393777323635580607144898021248152058,249955563744089122721854621298071612254],"VoterFromVoterParent"],[[224393777323635580607144898021248152058,249955563744089122721854621298071612254],[12937800391536231908324647127638186520,66280795779544227376177073920337078425],"VoterToVoterParent"],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[332262255414024854421745231429345981460,51492718212902479403695321501571662041],"VoterFromProposerParent"],[[332262255414024854421745231429345981460,51492718212902479403695321501571662041],[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"VoterToProposerParent"],[[125101350444128636945639187339273605295,310123101311045465600267532061795828904],[332262255414024854421745231429345981460,51492718212902479403695321501571662041],"VoterFromVoterParent"],[[332262255414024854421745231429345981460,51492718212902479403695321501571662041],[125101350444128636945639187339273605295,310123101311045465600267532061795828904],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[262616985261255455844155213193433237774,145446227922853493054259802531988219538],"VoterFromProposerParentAndVote"],[[262616985261255455844155213193433237774,145446227922853493054259802531988219538],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerParentAndVote"],[[209247141366778780182390572487884510544,72710930124973944574458366969623903493],[262616985261255455844155213193433237774,145446227922853493054259802531988219538],"VoterFromVoterParent"],[[262616985261255455844155213193433237774,145446227922853493054259802531988219538],[209247141366778780182390572487884510544,72710930124973944574458366969623903493],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[322067858235223357571533356548034754993,25038979323535615458325706946963617337],"VoterFromProposerParentAndVote"],[[322067858235223357571533356548034754993,25038979323535615458325706946963617337],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerParentAndVote"],[[169582566327277687462980578090338146030,202615754517709453024748782759834084329],[322067858235223357571533356548034754993,25038979323535615458325706946963617337],"VoterFromVoterParent"],[[322067858235223357571533356548034754993,25038979323535615458325706946963617337],[169582566327277687462980578090338146030,202615754517709453024748782759834084329],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[291484217031466680099610379240769034740,14200991401240643539941737876315122285],"VoterFromProposerParentAndVote"],[[291484217031466680099610379240769034740,14200991401240643539941737876315122285],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerParentAndVote"],[[136624341944058342936762535411176582562,123747836904407515690884885245025584803],[291484217031466680099610379240769034740,14200991401240643539941737876315122285],"VoterFromVoterParent"],[[291484217031466680099610379240769034740,14200991401240643539941737876315122285],[136624341944058342936762535411176582562,123747836904407515690884885245025584803],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[250457325780637301921996917305244435950,131586990515645526255840592907086140716],"TransactionFromProposerParent"],[[250457325780637301921996917305244435950,131586990515645526255840592907086140716],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"TransactionToProposerParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[125789532231136834849054141195108063588,176375398954734537356664169541815743441],"TransactionFromProposerParent"],[[125789532231136834849054141195108063588,176375398954734537356664169541815743441],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"TransactionToProposerParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"ProposerFromProposerParent"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"ProposerToProposerParent"],[[250457325780637301921996917305244435950,131586990515645526255840592907086140716],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],{"ProposerFromTransactionReference":0}],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[250457325780637301921996917305244435950,131586990515645526255840592907086140716],{"ProposerToTransactionReference":0}],[[125789532231136834849054141195108063588,176375398954734537356664169541815743441],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],{"ProposerFromTransactionReference":1}],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[125789532231136834849054141195108063588,176375398954734537356664169541815743441],{"ProposerToTransactionReference":1}],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[165849206361814032005123075275622624953,292716923126131207032776295167996351627],"VoterFromProposerParentAndVote"],[[165849206361814032005123075275622624953,292716923126131207032776295167996351627],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[302252672319357169528154936146331567083,122555437337793757347265884757286863129],[165849206361814032005123075275622624953,292716923126131207032776295167996351627],"VoterFromVoterParent"],[[165849206361814032005123075275622624953,292716923126131207032776295167996351627],[302252672319357169528154936146331567083,122555437337793757347265884757286863129],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[165849206361814032005123075275622624953,292716923126131207032776295167996351627],"VoterFromProposerVote"],[[165849206361814032005123075275622624953,292716923126131207032776295167996351627],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerVote"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[252531051328709822267746914084618967033,65712395607121182670459771704525383053],"VoterFromProposerParentAndVote"],[[252531051328709822267746914084618967033,65712395607121182670459771704525383053],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[224393777323635580607144898021248152058,249955563744089122721854621298071612254],[252531051328709822267746914084618967033,65712395607121182670459771704525383053],"VoterFromVoterParent"],[[252531051328709822267746914084618967033,65712395607121182670459771704525383053],[224393777323635580607144898021248152058,249955563744089122721854621298071612254],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[252531051328709822267746914084618967033,65712395607121182670459771704525383053],"VoterFromProposerVote"],[[252531051328709822267746914084618967033,65712395607121182670459771704525383053],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerVote"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[189249991685803379293848535722262871342,337821763772615626754419387951611947989],"VoterFromProposerParentAndVote"],[[189249991685803379293848535722262871342,337821763772615626754419387951611947989],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[332262255414024854421745231429345981460,51492718212902479403695321501571662041],[189249991685803379293848535722262871342,337821763772615626754419387951611947989],"VoterFromVoterParent"],[[189249991685803379293848535722262871342,337821763772615626754419387951611947989],[332262255414024854421745231429345981460,51492718212902479403695321501571662041],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[189249991685803379293848535722262871342,337821763772615626754419387951611947989],"VoterFromProposerVote"],[[189249991685803379293848535722262871342,337821763772615626754419387951611947989],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerVote"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[231129981218506605978178945671521672651,189797112381595739672591243032230990834],"VoterFromProposerParentAndVote"],[[231129981218506605978178945671521672651,189797112381595739672591243032230990834],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[262616985261255455844155213193433237774,145446227922853493054259802531988219538],[231129981218506605978178945671521672651,189797112381595739672591243032230990834],"VoterFromVoterParent"],[[231129981218506605978178945671521672651,189797112381595739672591243032230990834],[262616985261255455844155213193433237774,145446227922853493054259802531988219538],"VoterToVoterParent"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[31136538839648085434819171234342287861,261388421557995633548572008987764860601],"VoterFromProposerParentAndVote"],[[31136538839648085434819171234342287861,261388421557995633548572008987764860601],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[322067858235223357571533356548034754993,25038979323535615458325706946963617337],[31136538839648085434819171234342287861,261388421557995633548572008987764860601],"VoterFromVoterParent"],[[31136538839648085434819171234342287861,261388421557995633548572008987764860601],[322067858235223357571533356548034754993,25038979323535615458325706946963617337],"VoterToVoterParent"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[47376163009546864484697487729096902875,214868377026504667564962811517813785992],"VoterFromProposerParentAndVote"],[[47376163009546864484697487729096902875,214868377026504667564962811517813785992],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[291484217031466680099610379240769034740,14200991401240643539941737876315122285],[47376163009546864484697487729096902875,214868377026504667564962811517813785992],"VoterFromVoterParent"],[[47376163009546864484697487729096902875,214868377026504667564962811517813785992],[291484217031466680099610379240769034740,14200991401240643539941737876315122285],"VoterToVoterParent"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[171869182663025719558262817506489337178,311571485559242294165058830370527131644],"VoterFromProposerParentAndVote"],[[171869182663025719558262817506489337178,311571485559242294165058830370527131644],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[218716363703852801541862396078244676608,221872901472969775451833903017499552185],[171869182663025719558262817506489337178,311571485559242294165058830370527131644],"VoterFromVoterParent"],[[171869182663025719558262817506489337178,311571485559242294165058830370527131644],[218716363703852801541862396078244676608,221872901472969775451833903017499552185],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[171869182663025719558262817506489337178,311571485559242294165058830370527131644],"VoterFromProposerVote"],[[171869182663025719558262817506489337178,311571485559242294165058830370527131644],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerVote"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[328878694451253749819887723035603697049,86787972331288123207577812493295961288],"VoterFromProposerParentAndVote"],[[328878694451253749819887723035603697049,86787972331288123207577812493295961288],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[216764452460743606311030478227486484923,106930846498423695235466046775012050368],[328878694451253749819887723035603697049,86787972331288123207577812493295961288],"VoterFromVoterParent"],[[328878694451253749819887723035603697049,86787972331288123207577812493295961288],[216764452460743606311030478227486484923,106930846498423695235466046775012050368],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[328878694451253749819887723035603697049,86787972331288123207577812493295961288],"VoterFromProposerVote"],[[328878694451253749819887723035603697049,86787972331288123207577812493295961288],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerVote"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[271943753414996203885898299957183676473,291867039897861941128894107419356342652],"VoterFromProposerParentAndVote"],[[271943753414996203885898299957183676473,291867039897861941128894107419356342652],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[104447683660539520465595759369635189633,281072176520243182634918095165554116675],[271943753414996203885898299957183676473,291867039897861941128894107419356342652],"VoterFromVoterParent"],[[271943753414996203885898299957183676473,291867039897861941128894107419356342652],[104447683660539520465595759369635189633,281072176520243182634918095165554116675],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[271943753414996203885898299957183676473,291867039897861941128894107419356342652],"VoterFromProposerVote"],[[271943753414996203885898299957183676473,291867039897861941128894107419356342652],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerVote"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[166652751938431163656014158064412092315,246750559865952936766106253800759971456],"VoterFromProposerParentAndVote"],[[166652751938431163656014158064412092315,246750559865952936766106253800759971456],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"VoterToProposerParentAndVote"],[[309479661309386212115965602171518273580,312006182838606330110754301785646750660],[166652751938431163656014158064412092315,246750559865952936766106253800759971456],"VoterFromVoterParent"],[[166652751938431163656014158064412092315,246750559865952936766106253800759971456],[309479661309386212115965602171518273580,312006182838606330110754301785646750660],"VoterToVoterParent"],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],[166652751938431163656014158064412092315,246750559865952936766106253800759971456],"VoterFromProposerVote"],[[166652751938431163656014158064412092315,246750559865952936766106253800759971456],[239641839730766700995096596724873376432,285098089647576327019598725206922166228],"VoterToProposerVote"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[208338758569332088884389424917992202116,272827761987439643655066125485307543666],"TransactionFromProposerParent"],[[208338758569332088884389424917992202116,272827761987439643655066125485307543666],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"TransactionToProposerParent"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[198303767579166669377720876767768019014,274936154693994097795487386624187969091],"TransactionFromProposerParent"],[[198303767579166669377720876767768019014,274936154693994097795487386624187969091],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"TransactionToProposerParent"],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],[73149418366316677371068114780172198243,89204901837632810502081189591511820278],"ProposerFromProposerParent"],[[73149418366316677371068114780172198243,89204901837632810502081189591511820278],[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"ProposerToProposerParent"],[[208338758569332088884389424917992202116,272827761987439643655066125485307543666],[73149418366316677371068114780172198243,89204901837632810502081189591511820278],{"ProposerFromTransactionReference":0}],[[73149418366316677371068114780172198243,89204901837632810502081189591511820278],[208338758569332088884389424917992202116,272827761987439643655066125485307543666],{"ProposerToTransactionReference":0}],[[198303767579166669377720876767768019014,274936154693994097795487386624187969091],[73149418366316677371068114780172198243,89204901837632810502081189591511820278],{"ProposerFromTransactionReference":1}],[[73149418366316677371068114780172198243,89204901837632810502081189591511820278],[198303767579166669377720876767768019014,274936154693994097795487386624187969091],{"ProposerToTransactionReference":1}]],"prop_nodes":[[[0,0]],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503]],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],[334036293486207965471121892248981158011,119198624920124930478378759960496660278]],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228]],[[123968104178931226042920437395337340979,203856127139551421671955436744310702909]],[[73149418366316677371068114780172198243,89204901837632810502081189591511820278]]],"leader_nodes":{"1":[212379610789595152772554813072772936836,1250295795854552916842880354991653503],"0":[0,0],"4":[123968104178931226042920437395337340979,203856127139551421671955436744310702909],"2":[185584108269366270101306243932314476605,75516242955770381901310240491683559154],"3":[239641839730766700995096596724873376432,285098089647576327019598725206922166228]},"voter_chain_best_blocks":[[165849206361814032005123075275622624953,292716923126131207032776295167996351627],[252531051328709822267746914084618967033,65712395607121182670459771704525383053],[189249991685803379293848535722262871342,337821763772615626754419387951611947989],[231129981218506605978178945671521672651,189797112381595739672591243032230990834],[31136538839648085434819171234342287861,261388421557995633548572008987764860601],[47376163009546864484697487729096902875,214868377026504667564962811517813785992],[171869182663025719558262817506489337178,311571485559242294165058830370527131644],[328878694451253749819887723035603697049,86787972331288123207577812493295961288],[271943753414996203885898299957183676473,291867039897861941128894107419356342652],[166652751938431163656014158064412092315,246750559865952936766106253800759971456]],"pool_unconfirmed":[[198303767579166669377720876767768019014,274936154693994097795487386624187969091],[208338758569332088884389424917992202116,272827761987439643655066125485307543666]],"pool_ordered":[[16309190651391274726615603873253518193,74615886398085577132179308356420263538],[203416634904130912947837948206129956701,84667328377361762844812902622586151209],[127631816784194221745890895948874516060,98040460094865379368012538286735519919],[13666062443490743129725895377544875000,59332087016073312160008192530063055542],[113140849040239369915023536159678568992,215910713632893116095956307163399606234],[287097079470211302833715110561734626650,25926857114035119223559103865408553040],[112843536655743373806773716133998813328,299687879528442309170905353923949392757],[70492910119632306246052673805011834441,53517089152114679239997810090471361907],[135234043505129978719737467984837497597,174040853874324827229657708681579943514],[188852550260518993802415706584420624711,301812328892229109507515401587485851177],[169364742848526736697216141342124312999,126145157880518463053192077034749149436],[196935908130825214331027925217695769814,141572546089812697211828151672257505424],[192980804815934475414241153111321128041,67988076356156252877453577212225517225],[338233728719313876516491589050094215276,57721423586908637536166780724712872734],[250457325780637301921996917305244435950,131586990515645526255840592907086140716],[125789532231136834849054141195108063588,176375398954734537356664169541815743441]],"pool_unreferred":[[70492910119632306246052673805011834441,53517089152114679239997810090471361907],[127631816784194221745890895948874516060,98040460094865379368012538286735519919],[112843536655743373806773716133998813328,299687879528442309170905353923949392757],[169364742848526736697216141342124312999,126145157880518463053192077034749149436],[13666062443490743129725895377544875000,59332087016073312160008192530063055542],[196935908130825214331027925217695769814,141572546089812697211828151672257505424],[208338758569332088884389424917992202116,272827761987439643655066125485307543666],[16309190651391274726615603873253518193,74615886398085577132179308356420263538],[338233728719313876516491589050094215276,57721423586908637536166780724712872734],[250457325780637301921996917305244435950,131586990515645526255840592907086140716],[113140849040239369915023536159678568992,215910713632893116095956307163399606234],[203416634904130912947837948206129956701,84667328377361762844812902622586151209],[192980804815934475414241153111321128041,67988076356156252877453577212225517225],[135234043505129978719737467984837497597,174040853874324827229657708681579943514],[188852550260518993802415706584420624711,301812328892229109507515401587485851177],[198303767579166669377720876767768019014,274936154693994097795487386624187969091],[287097079470211302833715110561734626650,25926857114035119223559103865408553040],[125789532231136834849054141195108063588,176375398954734537356664169541815743441]],"proposer_node_data":[[[123968104178931226042920437395337340979,203856127139551421671955436744310702909],4,"Leader",10],[[185584108269366270101306243932314476605,75516242955770381901310240491683559154],2,"Leader",7],[[239641839730766700995096596724873376432,285098089647576327019598725206922166228],3,"Leader",10],[[0,0],0,"Leader",10],[[334036293486207965471121892248981158011,119198624920124930478378759960496660278],2,"NotLeaderAndConfirmed",3],[[212379610789595152772554813072772936836,1250295795854552916842880354991653503],1,"Leader",10],[[73149418366316677371068114780172198243,89204901837632810502081189591511820278],5,"PotentialLeader",0]],"voter_node_data":[[[48201290777533410771139438958502714464,27213940389256323001680455193029967216],3,1,"OnMainChain"],[[0,8],7,0,"OnMainChain"],[[231129981218506605978178945671521672651,189797112381595739672591243032230990834],3,4,"OnMainChain"],[[104447683660539520465595759369635189633,281072176520243182634918095165554116675],8,2,"OnMainChain"],[[216764452460743606311030478227486484923,106930846498423695235466046775012050368],7,2,"OnMainChain"],[[0,6],5,0,"OnMainChain"],[[332262255414024854421745231429345981460,51492718212902479403695321501571662041],2,3,"OnMainChain"],[[189249991685803379293848535722262871342,337821763772615626754419387951611947989],2,4,"OnMainChain"],[[166652751938431163656014158064412092315,246750559865952936766106253800759971456],9,3,"OnMainChain"],[[328878694451253749819887723035603697049,86787972331288123207577812493295961288],7,3,"OnMainChain"],[[243302063577874611527669088163318764758,83471547100651815018813919161061732632],8,1,"OnMainChain"],[[0,9],8,0,"OnMainChain"],[[0,10],9,0,"OnMainChain"],[[0,2],1,0,"OnMainChain"],[[211196780309865720881828649784815609313,45125330659475175485776413142186369343],5,1,"OnMainChain"],[[209247141366778780182390572487884510544,72710930124973944574458366969623903493],3,2,"OnMainChain"],[[171869182663025719558262817506489337178,311571485559242294165058830370527131644],6,3,"OnMainChain"],[[322067858235223357571533356548034754993,25038979323535615458325706946963617337],4,3,"OnMainChain"],[[287043098010830812125523790819448648783,131487889733446119788561996676586206793],1,1,"OnMainChain"],[[47376163009546864484697487729096902875,214868377026504667564962811517813785992],5,4,"OnMainChain"],[[136624341944058342936762535411176582562,123747836904407515690884885245025584803],5,2,"OnMainChain"],[[125101350444128636945639187339273605295,310123101311045465600267532061795828904],2,2,"OnMainChain"],[[262339210057093813629912155764713464083,4128170806167588733999200694047488729],6,1,"OnMainChain"],[[218716363703852801541862396078244676608,221872901472969775451833903017499552185],6,2,"OnMainChain"],[[12937800391536231908324647127638186520,66280795779544227376177073920337078425],1,2,"OnMainChain"],[[291484217031466680099610379240769034740,14200991401240643539941737876315122285],5,3,"OnMainChain"],[[0,7],6,0,"OnMainChain"],[[17914026366725359034877494502710405716,219442844074318402922155726089159906818],2,1,"OnMainChain"],[[165849206361814032005123075275622624953,292716923126131207032776295167996351627],0,4,"OnMainChain"],[[0,1],0,0,"OnMainChain"],[[334952087411119706473059337547418793746,229657452203028483274315913153769090502],4,1,"OnMainChain"],[[271943753414996203885898299957183676473,291867039897861941128894107419356342652],8,3,"OnMainChain"],[[0,4],3,0,"OnMainChain"],[[196222927150313727623849423392830757607,48601561847489824114585381840201303570],0,1,"OnMainChain"],[[302252672319357169528154936146331567083,122555437337793757347265884757286863129],0,3,"OnMainChain"],[[133620542141301101600462131143722368744,154310728365235349425042104278799319053],7,1,"OnMainChain"],[[0,3],2,0,"OnMainChain"],[[224393777323635580607144898021248152058,249955563744089122721854621298071612254],1,3,"OnMainChain"],[[0,5],4,0,"OnMainChain"],[[169582566327277687462980578090338146030,202615754517709453024748782759834084329],4,2,"OnMainChain"],[[193851307777460516918356531339904192027,165125033430926100122698488439366290008],0,2,"OnMainChain"],[[217358329842609508847039961830287659919,133479465929730160242868124882031279602],9,1,"OnMainChain"],[[252531051328709822267746914084618967033,65712395607121182670459771704525383053],1,4,"OnMainChain"],[[31136538839648085434819171234342287861,261388421557995633548572008987764860601],4,4,"OnMainChain"],[[309479661309386212115965602171518273580,312006182838606330110754301785646750660],9,2,"OnMainChain"],[[262616985261255455844155213193433237774,145446227922853493054259802531988219538],3,3,"OnMainChain"]]}
'''

In [5]:
dwg = svgwrite.Drawing('test.svg', profile='full')
#style part
dwg.defs.add(dwg.style('''rect.Proposer {
        fill:   #8FBAE1;
        stroke: #000000;
        stroke-width:0;
    }'''))
dwg.defs.add(dwg.style('''rect.ProposerLeader {
        fill:   #8FBAE1;
        stroke: #000000;
        stroke-width:2;
    }'''))

m=dwg.marker(id_='ToProposerParent', markerWidth="10",markerHeight="10", refX="0", refY="3", orient="auto", markerUnits="strokeWidth")
m.add(dwg.path(d="M0,0 L0,6 L{},3 z".format(ARROW_HEIGHT),fill="#000000"))
dwg.defs.add(m)

#parsing
bc = parseBlockchainDump(dump)
leader_set = bc['leader_set']
prop_votes = bc['prop_votes']
def propInfo(node, bc):
    ret = ['Proposer Block: {}'.format(node),
    'Leader: {}'.format(node in bc['leader_set']),
    'Votes: {}'.format(prop_votes[node])]
    for (h1, h2), edge in bc['edges'].items():
        if h1==node:
            if edge=='ProposerToProposerParent':
                ret.append('Proposer Parent: {}'.format(h2))
    return ',\n'.join(ret)

#drawing
for level, layer in enumerate(bc['prop_nodes']):
    exist_leader = 0
    for node in layer:
        if node in leader_set:
            addPropBlock(dwg,0,(PROP_BLOCK_HEIGHT+PROP_BLOCK_MARGIN)*level, leader=True, text=node[:6], info=propInfo(node,bc))
            exist_leader = 1
            break
    width = exist_leader
    for node in layer:
        if node not in leader_set:
            addPropBlock(dwg,(PROP_BLOCK_WIDTH+PROP_LEFT_MARGIN)*width,(PROP_BLOCK_HEIGHT+PROP_BLOCK_MARGIN)*level, leader=False, text=node[:6], info=propInfo(node,bc))
            width+=1
dwg.save()